In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from torchsummary import summary

In [2]:
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

])

In [3]:
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform = transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform = transform)

100%|██████████| 170M/170M [00:12<00:00, 13.5MB/s]


In [4]:
batch_size = 128
num_class = 10
epochs = 5
learning_rate = 0.001
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [14]:

model = models.mobilenet_v2(pretrained=True)

for param in model.parameters():
    param.requires_grad = False
num_ftrs = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_ftrs, num_class)




/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth
100%|██████████| 13.6M/13.6M [00:00<00:00, 122MB/s]


In [15]:
print(model.classifier)

Sequential(
  (0): Dropout(p=0.2, inplace=False)
  (1): Linear(in_features=1280, out_features=10, bias=True)
)


In [16]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
summary(model, (3,224, 224))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 112, 112]             864
       BatchNorm2d-2         [-1, 32, 112, 112]              64
             ReLU6-3         [-1, 32, 112, 112]               0
            Conv2d-4         [-1, 32, 112, 112]             288
       BatchNorm2d-5         [-1, 32, 112, 112]              64
             ReLU6-6         [-1, 32, 112, 112]               0
            Conv2d-7         [-1, 16, 112, 112]             512
       BatchNorm2d-8         [-1, 16, 112, 112]              32
  InvertedResidual-9         [-1, 16, 112, 112]               0
           Conv2d-10         [-1, 96, 112, 112]           1,536
      BatchNorm2d-11         [-1, 96, 112, 112]             192
            ReLU6-12         [-1, 96, 112, 112]               0
           Conv2d-13           [-1, 96, 56, 56]             864
      BatchNorm2d-14           [-1, 96,

In [17]:
optimizer = optim.Adam(model.classifier[1].parameters(), lr=learning_rate)

citerion = nn.CrossEntropyLoss()

def train_model(model, train_loader, epochs):
  model.train()
  for epoch in range(epochs):
    total_running_loss = 0
    correct = 0
    for input, label in train_loader:
      input, label = input.to(device), label.to(device)
      optimizer.zero_grad()
      output = model(input)
      loss = citerion(output, label)
      loss.backward()
      optimizer.step()
      total_running_loss += loss.item() * input.size(0)
      pred = output.argmax(dim=1)
      correct += pred.eq(label).sum().item()
    avg_loss = total_running_loss / len(train_loader.dataset)
    accuracy = correct / len(train_loader.dataset)
    print(f'Epoch: [{epoch+1}/{epochs}], Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}')


train_model(model, train_loader, epochs)

Epoch: [1/5], Loss: 0.9036, Accuracy: 0.7008
Epoch: [2/5], Loss: 0.7052, Accuracy: 0.7528
Epoch: [3/5], Loss: 0.6897, Accuracy: 0.7609
Epoch: [4/5], Loss: 0.6729, Accuracy: 0.7650
Epoch: [5/5], Loss: 0.6673, Accuracy: 0.7664


In [18]:
def evaluate_model (model, test_loader):
  model.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for input, label in test_loader:
      input, label = input.to(device), label.to(device)
      output = model(input)
      loss = citerion(output, label)
      test_loss += loss.item() * input.size(0)
      pred = output.argmax(dim=1)
      correct += pred.eq(label).sum().item()
    avg_loss = test_loss / len(test_loader.dataset)
    accuracy  = correct / len(test_loader.dataset)
    print(f'Test Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}')

evaluate_model(model, test_loader)

Test Loss: 0.6329, Accuracy: 0.7779
